# Saturday, November 8, 2025

# 1. IMPORT LIBRARIES

In [1]:
import nfl_data_py as nfl
import pandas as pd

# 2. LOAD THE DATASET
####  Load all REGULAR seasons only

In [41]:
# we want all regular season from 1999 to 2025
seasons = [2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017,
           2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008,
           2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999]
regular = True

# load play-by-play data
dataset = nfl.import_pbp_data(seasons, downcast=True, cache=False)

# keep regular season only
if regular and "season_type" in dataset.columns:
    dataset = dataset[dataset["season_type"] == "REG"]

2025 done.
2024 done.
2023 done.
2022 done.
2021 done.
2020 done.
2019 done.
2018 done.
2017 done.
2016 done.
2015 done.
2014 done.
2013 done.
2012 done.
2011 done.
2010 done.
2009 done.
2008 done.
2007 done.
2006 done.
2005 done.
2004 done.
2003 done.
2002 done.
2001 done.
2000 done.
1999 done.
Downcasting floats.


- Save the dataset for the first time
- All other times just load the dataset

In [42]:
# save the dataset
dataset.to_csv('NFL[1999-2025].csv')

In [43]:
# load our saved dataset
dataset = pd.read_csv('NFL[1999-2025].csv', low_memory=False)

# 3. CLEAN THE DATASET
## Offensive Metrics:
### [ Off_EPA | DSR | Off_ DVOA | AY/A | QB_EPA ]

#### Off_EPA:
- Offensive Expected Points Added by the possession team
#### DSR:
- Drive Success Rate = number_of_drive_first_downs / number_of_drives
#### Off_DVOA:
- Offensive Defense-Adjusted Value Over Average = (team_off_epa - league_off_epa) / absolute_value_of(league_off_epa)
#### AY/A:
- Adjusted Yards Per Attempt = (number_of_passing_yards + 20 * number_of_pass_touchdowns - 45 * number_of_interceptions) / number_of_pass_attempts
#### QB_EPA:
- Quarter Back's Expected Points Added by the possesion team

## Defensive Metrics:
### [ Def_EPA | Def_DVOA | NPRG ]
#### Def_EPA:
- Defensive Expected Points Allowed by the defense team
#### Def_DVOA:
- Defensive Defense-Adjusted Value Over Average = (team_def_epa - league_def_epa) / absolute_value_of(league_def_epa)
### NPRG:
- Net Pressure Rate Gained = number_of_times_qb_was_pressured / number_of_dropbacks

## Other Metrics
### [ Year | Week | Home_Team | Away_Team | Result ]
#### Year & Week:
- This model will have weights on each row of data to prioritize the latest games over outdated data (WIP. not implemented yet)
#### Home & Away Teams:
- Data will keep track of which teams were home and away for each game
#### Result:
- This is the boolean data we want to predict. True means home won, False means home lost

##### NOTE! All data will be based on Game-By-Game

### 3.1 Get The Offensive & Defensive Metrics

In [44]:
# keep only scrimmage plays that actually gain/lose yards: runs or passes
# (This implicitly excludes penalties-only, timeouts, spikes, kneels, punts, etc.)
play_mask = (dataset.get("rush", 0) == 1) | (dataset.get("pass", 0) == 1)
dataset = dataset[play_mask].copy()

# note! we'll calculate offensive and defensive DVOA later

# OFFENSE metrics (by possession team)
# epa: expected points added - how good offense is
offense = (
    # for each possession team in each game, get the average offensive metrics
    # we'll get the average of dsr and ay/a later
    dataset.groupby(["game_id", "posteam"])
       .agg(
           off_epa=("epa", "mean"), # average epa & dvoa
           qb_epa=("qb_epa", "mean"), # average QB epa
           # for dsr:
           # total drives and number of drive first downs
           total_drives=("drive", "sum"),
           successful_drives=("drive_first_downs", "count"),
           # for ay/a:
           # number of passing yards, pass touchdowns, interceptons, and pass attempts
           pass_yards=("passing_yards", "sum"),
           pass_touchdowns=("pass_touchdown", "sum"),
           interceptions=("interception", "sum"),
           pass_attempts=("pass_attempt", "sum")
       )
       .rename_axis(["game_id", "team"])
       .reset_index()
)

# DEFENSE metrics (what each defense allowed)
# epa_allowed: mean offensive EPA by opponents vs this defense - how weak defense is
defense = (
    # for each defense team in each game, get the average defensive metrics
    # we'll get the average of nprg later
    dataset.groupby(["game_id", "defteam"])
       .agg(
           def_epa=("epa", "mean"), # average epa & dvoa
           # for nprg:
           # total number of times QB was pressured and dropbacks
           pressures=("was_pressure", "count"), # true/false values. NaN/0.0
           dropbacks=("qb_dropback", "count")
       )
       .rename_axis(["game_id", "team"])
       .reset_index()
)

# combine the metrics we got from offense and defense
team_stats = offense.merge(defense, on=["game_id", "team"], how="outer")  # merge on 'team'
team_stats

,game_id,team,off_epa,qb_epa,total_drives,successful_drives,pass_yards,pass_touchdowns,interceptions,pass_attempts,def_epa,pressures,dropbacks
0,1999_01_ARI_PHI,ARI,-0.124676,-0.124676,1546.0,87,274.0,1.0,3.0,50.0,-0.250785,0,60
1,1999_01_ARI_PHI,PHI,-0.250785,-0.250785,941.0,61,91.0,2.0,2.0,30.0,-0.124676,0,87
2,1999_01_BUF_IND,BUF,-0.209526,-0.209526,834.0,68,300.0,1.0,2.0,47.0,0.103372,0,61
3,1999_01_BUF_IND,IND,0.103372,0.103372,699.0,62,284.0,2.0,2.0,33.0,-0.209526,0,68
4,1999_01_CAR_NO,CAR,-0.350204,-0.350204,914.0,56,207.0,1.0,1.0,39.0,-0.204681,0,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13679,2025_10_PHI_GB,PHI,-0.069291,-0.069291,593.0,63,183.0,1.0,0.0,26.0,-0.152826,0,67
13680,2025_10_PIT_LAC,LAC,-0.111648,-0.111648,938.0,72,220.0,1.0,0.0,38.0,-0.372950,0,52
13681,2025_10_PIT_LAC,PIT,-0.372950,-0.372950,696.0,52,161.0,1.0,2.0,34.0,-0.111648,0,72
13682,2025_11_NYJ_NE,NE,0.138452,0.138452,556.0,64,281.0,1.0,0.0,35.0,-0.085097,0,57


In [45]:
# it's time to find the averages of off_dvoa, def_dvoa, dsr, ay/a, and nprg
team_eff = team_stats[['game_id', 'team', 'off_epa', 'def_epa', 'qb_epa']].copy()

# METRIC CALCULATIONS
# DVOA (offensive and defensive) = (team_epa - league_epa) / abs(league_epa)
# calculate the offensive and defensive league epa first
off_league_epa = team_stats['off_epa'].mean()
def_league_epa = team_stats['def_epa'].mean()
# now calculate the offensive and defensive DVOA
team_eff['off_dvoa'] = (team_stats['off_epa'] - off_league_epa) / abs(off_league_epa)
team_eff['def_dvoa'] = (team_stats['def_epa'] - def_league_epa) / abs(def_league_epa)

# DSR = successful_drives / total_drives
team_eff['dsr'] = team_stats['successful_drives'] / team_stats['total_drives']

# AY/A = (passing_yards + 20 * pass_touchdowns - 45 * interceptions) / pass_attempts
team_eff['ay/a'] = (team_stats['pass_yards'] + 20 * team_stats['pass_touchdowns']
                    - 45 * team_stats['interceptions']) / team_stats['pass_attempts']

# NPRG = pressures / dropbacks
team_eff['nprg'] = team_stats['pressures'] / team_stats['dropbacks']

team_eff

,game_id,team,off_epa,def_epa,qb_epa,off_dvoa,def_dvoa,dsr,ay/a,nprg
0,1999_01_ARI_PHI,ARI,-0.124676,-0.250785,-0.124676,-8.238406,-17.582923,0.056274,3.180000,0.0
1,1999_01_ARI_PHI,PHI,-0.250785,-0.124676,-0.250785,-17.582923,-8.238406,0.064825,1.366667,0.0
2,1999_01_BUF_IND,BUF,-0.209526,0.103372,-0.209526,-14.525645,8.659734,0.081535,4.893617,0.0
3,1999_01_BUF_IND,IND,0.103372,-0.209526,0.103372,8.659734,-14.525645,0.088698,7.090909,0.0
4,1999_01_CAR_NO,CAR,-0.350204,-0.204681,-0.350204,-24.949785,-14.166638,0.061269,4.666667,0.0
...,...,...,...,...,...,...,...,...,...,...
13679,2025_10_PHI_GB,PHI,-0.069291,-0.152826,-0.069291,-4.134427,-10.324250,0.106239,7.807692,0.0
13680,2025_10_PIT_LAC,LAC,-0.111648,-0.372950,-0.111648,-7.273024,-26.635203,0.076759,6.315789,0.0
13681,2025_10_PIT_LAC,PIT,-0.372950,-0.111648,-0.372950,-26.635203,-7.273024,0.074713,2.676471,0.0
13682,2025_11_NYJ_NE,NE,0.138452,-0.085097,0.138452,11.259125,-5.305635,0.115108,8.600000,0.0


### 3.2. Get The Away & Home Teams

In [46]:
# split into away and home DataFrames
away = team_eff.iloc[::2].reset_index(drop=True)  # even indices → away team
home = team_eff.iloc[1::2].reset_index(drop=True)  # odd indices → home team

# merge side by side
gbg = pd.concat([away, home], axis=1, keys=['away', 'home'])

# flatten the MultiIndex columns (e.g., 'away_team', 'home_team')
gbg.columns = [f"{side}_{col}" for side, col in gbg.columns]

# keep only one copy of game_id
gbg = gbg.drop(columns=['home_game_id'])
gbg = gbg.rename(columns={'away_game_id': 'game_id'})

### 3.3. Get The Years And Weeks
- Not yet weighted (WIP)

In [47]:
# split game_id into parts
date = dataset[['game_id']].drop_duplicates(['game_id'])
date[['year', 'week', 'away', 'home']] = date['game_id'].str.split('_', expand=True, n=3)
date = date.sort_values('game_id', ascending=True).reset_index()
date['year'] = date['year'].astype(int)
date['week'] = date['week'].astype(int)
gbg = gbg.join(date[['year', 'week']])

gbg

,game_id,away_team,away_off_epa,away_def_epa,away_qb_epa,away_off_dvoa,away_def_dvoa,away_dsr,away_ay/a,away_nprg,...,home_off_epa,home_def_epa,home_qb_epa,home_off_dvoa,home_def_dvoa,home_dsr,home_ay/a,home_nprg,year,week
0,1999_01_ARI_PHI,ARI,-0.124676,-0.250785,-0.124676,-8.238406,-17.582923,0.056274,3.180000,0.0,...,-0.250785,-0.124676,-0.250785,-17.582923,-8.238406,0.064825,1.366667,0.0,1999,1
1,1999_01_BUF_IND,BUF,-0.209526,0.103372,-0.209526,-14.525645,8.659734,0.081535,4.893617,0.0,...,0.103372,-0.209526,0.103372,8.659734,-14.525645,0.088698,7.090909,0.0,1999,1
2,1999_01_CAR_NO,CAR,-0.350204,-0.204681,-0.350204,-24.949785,-14.166638,0.061269,4.666667,0.0,...,-0.204681,-0.350204,-0.204681,-14.166638,-24.949785,0.057093,5.846154,0.0,1999,1
3,1999_01_CIN_TEN,CIN,0.173752,0.039889,0.173752,13.874885,3.955703,0.080271,3.972973,0.0,...,0.039889,0.173752,0.122393,3.955703,13.874885,0.076709,10.470588,0.0,1999,1
4,1999_01_DAL_WAS,DAL,0.277992,0.200565,0.277992,21.598945,15.861709,0.065918,6.540000,0.0,...,0.200565,0.277992,0.200565,15.861709,21.598945,0.071992,11.722222,0.0,1999,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6837,2025_10_NO_CAR,CAR,-0.300061,0.142840,-0.300061,-21.234240,11.584328,0.113978,2.925926,0.0,...,0.142840,-0.300061,0.142840,11.584328,-21.234240,0.089779,10.733333,0.0,2025,10
6838,2025_10_NYG_CHI,CHI,0.076580,-0.005457,0.076580,6.674534,0.595623,0.091413,6.666667,0.0,...,-0.005457,0.076580,-0.005457,0.595623,6.674534,0.084687,7.175000,0.0,2025,10
6839,2025_10_PHI_GB,GB,-0.152826,-0.069291,-0.152826,-10.324250,-4.134427,0.091033,4.631579,0.0,...,-0.069291,-0.152826,-0.069291,-4.134427,-10.324250,0.106239,7.807692,0.0,2025,10
6840,2025_10_PIT_LAC,LAC,-0.111648,-0.372950,-0.111648,-7.273024,-26.635203,0.076759,6.315789,0.0,...,-0.372950,-0.111648,-0.372950,-26.635203,-7.273024,0.074713,2.676471,0.0,2025,10


### 3.4. Get The Game Results
##### 'Result' column will have True and False boolean values
- True = Home wins!
- False = Home loses...

In [48]:
# find the result of each game
result = dataset[['game_id', 'home_score', 'away_score']].drop_duplicates(['game_id'])
result['result'] = result['home_score'] > result['away_score']

# reset the indices to match the ones in gbg dataframe
result = result.sort_values('game_id', ascending=True).reset_index()

result

,index,game_id,home_score,away_score,result
0,1160281,1999_01_ARI_PHI,24,25,False
1,1160482,1999_01_BUF_IND,31,14,True
2,1160664,1999_01_CAR_NO,19,10,True
3,1160838,1999_01_CIN_TEN,36,35,True
4,1161027,1999_01_DAL_WAS,35,41,False
...,...,...,...,...,...
6837,24883,2025_10_NO_CAR,7,17,False
6838,25036,2025_10_NYG_CHI,24,20,True
6839,25214,2025_10_PHI_GB,7,10,False
6840,25386,2025_10_PIT_LAC,25,10,True


### 3.5. Tie It All Together

In [49]:
# join the result dataframe to our main dataframe
gbg = gbg.join(result[['result']])

# organize the dataframe
cols = ['year', 'week',
        'away_team',
        'away_off_epa', 'away_def_epa', 'away_qb_epa', 'away_off_dvoa', 'away_def_dvoa', 'away_dsr', 'away_ay/a', 'away_nprg',
        'home_team',
        'home_off_epa', 'home_def_epa', 'home_qb_epa', 'home_off_dvoa', 'home_def_dvoa', 'home_dsr', 'home_ay/a', 'home_nprg',
        'result']
gbg = gbg[cols]

gbg

,year,week,away_team,away_off_epa,away_def_epa,away_qb_epa,away_off_dvoa,away_def_dvoa,away_dsr,away_ay/a,...,home_team,home_off_epa,home_def_epa,home_qb_epa,home_off_dvoa,home_def_dvoa,home_dsr,home_ay/a,home_nprg,result
0,1999,1,ARI,-0.124676,-0.250785,-0.124676,-8.238406,-17.582923,0.056274,3.180000,...,PHI,-0.250785,-0.124676,-0.250785,-17.582923,-8.238406,0.064825,1.366667,0.0,False
1,1999,1,BUF,-0.209526,0.103372,-0.209526,-14.525645,8.659734,0.081535,4.893617,...,IND,0.103372,-0.209526,0.103372,8.659734,-14.525645,0.088698,7.090909,0.0,True
2,1999,1,CAR,-0.350204,-0.204681,-0.350204,-24.949785,-14.166638,0.061269,4.666667,...,NO,-0.204681,-0.350204,-0.204681,-14.166638,-24.949785,0.057093,5.846154,0.0,True
3,1999,1,CIN,0.173752,0.039889,0.173752,13.874885,3.955703,0.080271,3.972973,...,TEN,0.039889,0.173752,0.122393,3.955703,13.874885,0.076709,10.470588,0.0,True
4,1999,1,DAL,0.277992,0.200565,0.277992,21.598945,15.861709,0.065918,6.540000,...,WAS,0.200565,0.277992,0.200565,15.861709,21.598945,0.071992,11.722222,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6837,2025,10,CAR,-0.300061,0.142840,-0.300061,-21.234240,11.584328,0.113978,2.925926,...,NO,0.142840,-0.300061,0.142840,11.584328,-21.234240,0.089779,10.733333,0.0,False
6838,2025,10,CHI,0.076580,-0.005457,0.076580,6.674534,0.595623,0.091413,6.666667,...,NYG,-0.005457,0.076580,-0.005457,0.595623,6.674534,0.084687,7.175000,0.0,True
6839,2025,10,GB,-0.152826,-0.069291,-0.152826,-10.324250,-4.134427,0.091033,4.631579,...,PHI,-0.069291,-0.152826,-0.069291,-4.134427,-10.324250,0.106239,7.807692,0.0,False
6840,2025,10,LAC,-0.111648,-0.372950,-0.111648,-7.273024,-26.635203,0.076759,6.315789,...,PIT,-0.372950,-0.111648,-0.372950,-26.635203,-7.273024,0.074713,2.676471,0.0,True


In [50]:
# save the dataframe for the next steps
gbg.to_csv('V3_NFL_gbg_data[1999-2025].csv')

### 4. PREP FOR TRAINING
- The model should know EPA, YPP, home & away scores
- The home & away teams are only for our eyes, not the model

In [51]:
import pandas as pd

dataset = pd.read_csv('V3_NFL_gbg_data[1999-2025].csv')

In [52]:
team_map = {
    'NO': 1,
    'BUF': 2,
    'JAX': 3,
    'CLE': 4,
    'PHI': 5,
    'GB': 6,
    'LA': 7,
    'LAC': 8,
    'NE': 9,
    'IND': 10,
    'CHI': 11,
    'WAS': 12,
    'NYJ': 13,
    'SEA': 14,
    'ATL': 15,
    'DEN': 16,
    'MIN': 17,
    'ARI': 18,
    'DET': 19,
    'BAL': 20,
    'CIN': 21,
    'LV': 22,
    'TEN': 23,
    'MIA': 24,
    'DAL': 25,
    'KC': 26,
    'PIT': 27,
    'HOU': 28,
    'SF': 29,
    'CAR': 30,
    'NYG': 31,
    'TB': 32
}

dataset['away_team'] = dataset['away_team'].map(team_map)
dataset['home_team'] = dataset['home_team'].map(team_map)

# Removing unnamed columns using drop function
dataset.drop(dataset.columns[dataset.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

dataset

,year,week,away_team,away_off_epa,away_def_epa,away_qb_epa,away_off_dvoa,away_def_dvoa,away_dsr,away_ay/a,...,home_team,home_off_epa,home_def_epa,home_qb_epa,home_off_dvoa,home_def_dvoa,home_dsr,home_ay/a,home_nprg,result
0,1999,1,18,-0.124676,-0.250785,-0.124676,-8.238406,-17.582923,0.056274,3.180000,...,5,-0.250785,-0.124676,-0.250785,-17.582923,-8.238406,0.064825,1.366667,0.0,False
1,1999,1,2,-0.209526,0.103372,-0.209526,-14.525645,8.659734,0.081535,4.893617,...,10,0.103372,-0.209526,0.103372,8.659734,-14.525645,0.088698,7.090909,0.0,True
2,1999,1,30,-0.350204,-0.204681,-0.350204,-24.949785,-14.166638,0.061269,4.666667,...,1,-0.204681,-0.350204,-0.204681,-14.166638,-24.949785,0.057093,5.846154,0.0,True
3,1999,1,21,0.173752,0.039889,0.173752,13.874885,3.955703,0.080271,3.972973,...,23,0.039889,0.173752,0.122393,3.955703,13.874885,0.076709,10.470588,0.0,True
4,1999,1,25,0.277992,0.200565,0.277992,21.598945,15.861709,0.065918,6.540000,...,12,0.200565,0.277992,0.200565,15.861709,21.598945,0.071992,11.722222,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6837,2025,10,30,-0.300061,0.142840,-0.300061,-21.234240,11.584328,0.113978,2.925926,...,1,0.142840,-0.300061,0.142840,11.584328,-21.234240,0.089779,10.733333,0.0,False
6838,2025,10,11,0.076580,-0.005457,0.076580,6.674534,0.595623,0.091413,6.666667,...,31,-0.005457,0.076580,-0.005457,0.595623,6.674534,0.084687,7.175000,0.0,True
6839,2025,10,6,-0.152826,-0.069291,-0.152826,-10.324250,-4.134427,0.091033,4.631579,...,5,-0.069291,-0.152826,-0.069291,-4.134427,-10.324250,0.106239,7.807692,0.0,False
6840,2025,10,8,-0.111648,-0.372950,-0.111648,-7.273024,-26.635203,0.076759,6.315789,...,27,-0.372950,-0.111648,-0.372950,-26.635203,-7.273024,0.074713,2.676471,0.0,True


In [53]:
# set the training and testing variables
from sklearn.model_selection import train_test_split

X = dataset.drop(columns=['result'])
y = dataset['result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 80% train / 20% test

### 5. AI MODEL TRAINING


In [54]:
import xgboost as xgb
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
import cupy as cp

c:\Users\octav\Downloads\College\CS\CSUSB\TRAINING ARC\Mini Projects\NFL_AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
X = cp.asarray(X.values)
y = cp.asarray(y.values)

def objective(trial):
    param = {
        "max_depth": trial.suggest_int("max_depth", 3, 30),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
        # "tree_method": "hist",
        "device": "cuda",
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for tr, va in skf.split(
        cp.asnumpy(X), cp.asnumpy(y)
    ):  # indices on CPU are fine
        model = xgb.XGBClassifier(**param)
        model.fit(X[tr], y[tr])
        # GPU inference (no warning)
        p = model.predict_proba(X[va])[:, 1]
        # metric on CPU
        scores.append(roc_auc_score(cp.asnumpy(y[va]), cp.asnumpy(p)))
    return float(cp.asarray(scores).mean())

In [56]:
# Create and run the optimization process with 100 trials
study = optuna.create_study(study_name="xgboost_study_gpu_cupy", direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True, n_jobs=-1)

# Retrieve the best parameter values
best_params = study.best_params
print(f"\nBest parameters: {best_params}")

[I 2025-11-15 11:09:17,833] A new study created in memory with name: xgboost_study_gpu_cupy
Best trial: 7. Best value: 0.572794:   1%|          | 1/100 [00:14<24:05, 14.60s/it]

[I 2025-11-15 11:09:32,758] Trial 7 finished with value: 0.5727937881545699 and parameters: {'max_depth': 5, 'learning_rate': 0.04872264279994409, 'n_estimators': 101, 'subsample': 0.7153220182430504, 'colsample_bytree': 0.7831166250237821, 'min_child_weight': 5, 'gamma': 0.7060611490925989}. Best is trial 7 with value: 0.5727937881545699.


Best trial: 6. Best value: 0.573576:   2%|▏         | 2/100 [00:15<10:48,  6.62s/it]

[I 2025-11-15 11:09:33,785] Trial 6 finished with value: 0.5735763164071453 and parameters: {'max_depth': 4, 'learning_rate': 0.04199755871003895, 'n_estimators': 126, 'subsample': 0.7779706621436437, 'colsample_bytree': 0.7639753361732975, 'min_child_weight': 1, 'gamma': 1.7810490941097323}. Best is trial 6 with value: 0.5735763164071453.


Best trial: 5. Best value: 0.576744:   3%|▎         | 3/100 [00:31<17:46, 10.99s/it]

...

[I 2025-11-15 11:18:59,557] Trial 84 finished with value: 0.563731440978874 and parameters: {'max_depth': 24, 'learning_rate': 0.022421964040878595, 'n_estimators': 945, 'subsample': 0.6043189413342279, 'colsample_bytree': 0.6638910938940139, 'min_child_weight': 7, 'gamma': 0.036191323980012235}. Best is trial 99 with value: 0.586667608104909.


Best trial: 99. Best value: 0.586668:  99%|█████████▉| 99/100 [09:41<00:04,  4.80s/it]

[I 2025-11-15 11:19:00,146] Trial 89 finished with value: 0.5667882916972856 and parameters: {'max_depth': 13, 'learning_rate': 0.015270397422038187, 'n_estimators': 963, 'subsample': 0.6680692754200007, 'colsample_bytree': 0.7030431724445966, 'min_child_weight': 8, 'gamma': 0.5225637904080807}. Best is trial 99 with value: 0.586667608104909.


Best trial: 99. Best value: 0.586668: 100%|██████████| 100/100 [09:47<00:00,  5.88s/it]

[I 2025-11-15 11:19:05,841] Trial 98 finished with value: 0.5708138878279417 and parameters: {'max_depth': 10, 'learning_rate': 0.010323062530591186, 'n_estimators': 974, 'subsample': 0.6712132402154134, 'colsample_bytree': 0.9424122773880244, 'min_child_weight': 1, 'gamma': 0.8099582854926308}. Best is trial 99 with value: 0.586667608104909.

Best parameters: {'max_depth': 9, 'learning_rate': 0.01073286560105942, 'n_estimators': 383, 'subsample': 0.6658807352502142, 'colsample_bytree': 0.588249678172766, 'min_child_weight': 8, 'gamma': 3.669474432341215}


In [69]:
import numpy as np

# iterations:
# max_depth=14, learning_rate=0.010573629786541423, n_estimators=140, subsample=0.6926815018566853, colsample_bytree=0.6226063389661705, min_child_weight=4, gamma=2.9740841251220034   | 57.90%
# max_depth=30, learning_rate=0.03192091052817414, n_estimators=105, subsample=0.5270193265047416, colsample_bytree=0.5489363584973745, min_child_weight=4, gamma=3.733593582255768     | 58.45%
# max_depth=30, learning_rate=0.03192091052817414, n_estimators=1000, subsample=0.5270193265047416, colsample_bytree=0.5489363584973745, min_child_weight=4, gamma=3.733593582255768    | 57.46%
# max_depth=30, learning_rate=0.03192091052817414, n_estimators=105, subsample=0.6926815018566853, colsample_bytree=0.6226063389661705, min_child_weight=4, gamma=3.733593582255768     | 57.57%

classifier = xgb.XGBClassifier(max_depth=30, learning_rate=0.03192091052817414, n_estimators=105, subsample=0.5270193265047416, colsample_bytree=0.5489363584973745, min_child_weight=4, gamma=3.733593582255768)
classifier.fit(X_train, y_train)

# test the model
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



# cross-validation
from sklearn.model_selection import cross_val_score, KFold

# define number of k-folds
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# perform k-fold cross-validation
cross_val_results = cross_val_score(classifier, X_train, y_train, cv=kf)

# evaluation metrics
print("Cross-Validation Results (Accuracy): ")
for i, result in enumerate(cross_val_results, 1):
    print(f" Fold {i}: {result * 100:.2f}%")

print(f"Mean Accuracy: {cross_val_results.mean() * 100:.2f}%")

Accuracy: 0.581446311176041
Cross-Validation Results (Accuracy): 
 Fold 1: 57.17%
 Fold 2: 59.73%
 Fold 3: 55.80%
 Fold 4: 56.12%
 Fold 5: 55.58%
Mean Accuracy: 56.88%


In [70]:
import joblib

# save the model
joblib.dump(classifier, './V3_NFL[1999-2025].joblib')

['./V3_NFL[1999-2025].joblib']

#### 6. TEST THE MODEL WITH NEW DATA

In [ ]:
import joblib
import pandas as pd

# load the model
rf = joblib.load('./V3_NFL[1999-2025].joblib')

# load the dataset
df = pd.read_csv('V3_NFL_gbg_data[1999-2025].csv')
df.drop(
    df.columns[df.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True
    )

df

,year,week,away_team,away_off_epa,away_def_epa,away_qb_epa,away_off_dvoa,away_def_dvoa,away_dsr,away_ay/a,...,home_team,home_off_epa,home_def_epa,home_qb_epa,home_off_dvoa,home_def_dvoa,home_dsr,home_ay/a,home_nprg,result
0,1999,1,ARI,-0.124676,-0.250785,-0.124676,-8.238406,-17.582923,0.056274,3.180000,...,PHI,-0.250785,-0.124676,-0.250785,-17.582923,-8.238406,0.064825,1.366667,0.0,False
1,1999,1,BUF,-0.209526,0.103372,-0.209526,-14.525645,8.659734,0.081535,4.893617,...,IND,0.103372,-0.209526,0.103372,8.659734,-14.525645,0.088698,7.090909,0.0,True
2,1999,1,CAR,-0.350204,-0.204681,-0.350204,-24.949785,-14.166638,0.061269,4.666667,...,NO,-0.204681,-0.350204,-0.204681,-14.166638,-24.949785,0.057093,5.846154,0.0,True
3,1999,1,CIN,0.173752,0.039889,0.173752,13.874885,3.955703,0.080271,3.972973,...,TEN,0.039889,0.173752,0.122393,3.955703,13.874885,0.076709,10.470588,0.0,True
4,1999,1,DAL,0.277992,0.200565,0.277992,21.598945,15.861709,0.065918,6.540000,...,WAS,0.200565,0.277992,0.200565,15.861709,21.598945,0.071992,11.722222,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6837,2025,10,CAR,-0.300061,0.142840,-0.300061,-21.234240,11.584328,0.113978,2.925926,...,NO,0.142840,-0.300061,0.142840,11.584328,-21.234240,0.089779,10.733333,0.0,False
6838,2025,10,CHI,0.076580,-0.005457,0.076580,6.674534,0.595623,0.091413,6.666667,...,NYG,-0.005457,0.076580,-0.005457,0.595623,6.674534,0.084687,7.175000,0.0,True
6839,2025,10,GB,-0.152826,-0.069291,-0.152826,-10.324250,-4.134427,0.091033,4.631579,...,PHI,-0.069291,-0.152826,-0.069291,-4.134427,-10.324250,0.106239,7.807692,0.0,False
6840,2025,10,LAC,-0.111648,-0.372950,-0.111648,-7.273024,-26.635203,0.076759,6.315789,...,PIT,-0.372950,-0.111648,-0.372950,-26.635203,-7.273024,0.074713,2.676471,0.0,True


In [16]:
# dictionary for team name mapping. team name -> number for the model's interpretation
team_map = {
    'NO': 1,
    'BUF': 2,
    'JAX': 3,
    'CLE': 4,
    'PHI': 5,
    'GB': 6,
    'LA': 7,
    'LAC': 8,
    'NE': 9,
    'IND': 10,
    'CHI': 11,
    'WAS': 12,
    'NYJ': 13,
    'SEA': 14,
    'ATL': 15,
    'DEN': 16,
    'MIN': 17,
    'ARI': 18,
    'DET': 19,
    'BAL': 20,
    'CIN': 21,
    'LV': 22,
    'TEN': 23,
    'MIA': 24,
    'DAL': 25,
    'KC': 26,
    'PIT': 27,
    'HOU': 28,
    'SF': 29,
    'CAR': 30,
    'NYG': 31,
    'TB': 32
}

# the values we want to test: year, week, home & away teams
year, week           = 2025 ,  11
away_team, home_team = 'DAL', 'LV'
# columns needed to find the average of these metrics for both teams
cols = ['off_epa', 'def_epa', 'qb_epa', 'off_dvoa', 'def_dvoa', 'dsr', 'ay/a', 'nprg']
# max last number of games
n = 5

i = 1
while i <= n:
    # GET THE AVERAGES OF THE METRICS FROM THE LAST N GAMES
    # look at columns where this team is either away or home, whichever is most recent for last n games
    # away team
    away_data = df[(df['away_team'] == away_team) | (df['home_team'] == away_team)].tail(i)
    away_data = away_data[[f'away_{col}' for col in cols]].mean()

    # home team
    home_data = df[(df['away_team'] == home_team) | (df['home_team'] == home_team)].tail(i)
    home_data = home_data[[f'away_{col}' for col in cols]].mean()

    # CREATE A DICTIONARY TO HOLD THE NEW DATA
    new_data = {
        'year': year, 'week': week,
        # away stats
        'away_team': team_map[away_team],
        'away_off_epa': 0, 'away_def_epa': 0, 'away_qb_epa': 0,
        'away_off_dvoa': 0, 'away_def_dvoa': 0,
        'away_dsr': 0, 'away_ay/a': 0, 'away_nprg': 0,
        # home stats
        'home_team': team_map[home_team],
        'home_off_epa': 0, 'home_def_epa': 0, 'home_qb_epa': 0,
        'home_off_dvoa': 0, 'home_def_dvoa': 0,
        'home_dsr': 0, 'home_ay/a': 0, 'home_nprg': 0,
    }

    # assign the metrics to new data dictionary
    for col, metric in away_data.items():
        new_data[col] = metric
    for col, metric in home_data.items():
        new_data[col] = metric

    # create the new dataframe
    new_X = pd.DataFrame(data=new_data, index=[0])

    # PREDICT THE NEW VALUES
    pred = rf.predict(new_X)
    if pred == 1:
        print(f"{home_team} will win! Based on last {i} games.")
    else:
        print(f"{away_team} will win! Based on last {i} games.")

    i += 1

# W10: 61% -> 30% -> 42.8%
# W 9: 24% -> 50% -> 64.2%

LV will win! Based on last 1 games.
LV will win! Based on last 2 games.
DAL will win! Based on last 3 games.
DAL will win! Based on last 4 games.
DAL will win! Based on last 5 games.
